### 1. Import
Importing the SBML and running the "create_pathway" script and loading cobra and pandas.

In [1]:
# Importing used packages and model
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd
import matplotlib.pyplot as plt
model = read_sbml_model('iML1515.xml')
%run -i create_pathway_latest.ipynb

### 2. Defining media
The exchange reactions for substrate for all 3 pathways are added to the model their concentration in the medium is set.

In [3]:
model.add_boundary(model.metabolites.get_by_id("ncam_e"), type="exchange")
model.add_boundary(model.metabolites.get_by_id("nr_e"), type="exchange")
model.add_boundary(model.metabolites.get_by_id("na_e"), type="exchange")
model.reactions.EX_ncam_e.lower_bound=-10
model.reactions.EX_nr_e.lower_bound=-10
model.reactions.EX_na_e.lower_bound=-10
medium=model.medium
medium['EX_ncam_e'] = 10
medium['EX_nr_e'] = 10
medium["EX_na_e"] = 10
medium["EX_glc__D_e"] = 10
model.medium=medium

### 3. Table of media composition

In [4]:
print('Add table number and name')
# create a data frame to plot a table
med_names = dict()
for flux in medium:
    meta = model.reactions.get_by_id(flux).metabolites
    for element in meta:
        med_names[flux] = element.name
df_med = pd.DataFrame(medium, index=['uptake'])
df_names = pd.DataFrame(med_names, index=['name'])
display(pd.concat([df_names,df_med]))

Add table number and name


,EX_pi_e,EX_co2_e,EX_fe3_e,EX_h_e,EX_mn2_e,EX_fe2_e,EX_glc__D_e,EX_zn2_e,EX_mg2_e,EX_ca2_e,...,EX_nh4_e,EX_k_e,EX_na1_e,EX_cl_e,EX_o2_e,EX_tungs_e,EX_slnt_e,EX_ncam_e,EX_nr_e,EX_na_e
name,Phosphate,CO2 CO2,Iron (Fe3+),H+,Manganese,Fe2+ mitochondria,D-Glucose,Zinc,Magnesium,Calcium,...,Ammonium,Potassium,Sodium,Chloride,O2 O2,Tungstate,Selenite,nicotineamide,nicotineamide riboside,Nicotinic acid
uptake,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,10,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,10,10,10


In [6]:
model.metabolites.ncam_e

Metabolite identifier,ncam_e
Name,nicotineamide
Memory address,0x1bd121c3490
Formula,C6H6N2O
Compartment,e
In 1 reaction(s),EX_ncam_e


### Creating models for each pathway

In [7]:
# creating the models with the respective pathways
nam_model = create_pathway_latest(model.copy(),1)
na_model = create_pathway_latest(model.copy(),2)
nr_model = create_pathway_latest(model.copy(),3)
models = [model, (nam_model,'NAMPT'),(na_model,'NMN Synthetase'),(nr_model,'DeoD')]

NameError: name 'create_pathway_latest' is not defined

### Gene knockout of NMNDA

### Maximum biomass growth of the cell with the implemented pathways

In [5]:
with nam_model:
    growth = nam_model.optimize()
    max_growth_rate = growth.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
    print('NAM-pathway biomass growth', max_growth_rate,'mmol/gDW*h')
with na_model:
    growth = na_model.optimize()
    max_growth_rate = growth.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
    print('NA-pathway biomass growth', max_growth_rate,'mmol/gDW*h')
with nr_model:
    growth = nr_model.optimize()
    max_growth_rate = growth.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
    print('NR-pathway biomass growth', max_growth_rate,'mmol/gDW*h')

NAM-pathway biomass growth 1.3845699440948451 mmol/gDW*h
NA-pathway biomass growth 0.8769972144269672 mmol/gDW*h
NR-pathway biomass growth 14.48258966730981 mmol/gDW*h


### Max productivity and yield

In [6]:
with nam_model:
    print('With NAM pathway:')
    nam_model.objective = nam_model.reactions.NAMPT #Setting the objective to the reaction, which is producing NMN
    nmn_production = nam_model.optimize().objective_value
    print('Maximum productivity of NMN =', nmn_production, 'mmol/gDW*h')

    glc_uptake_flux = nam_model.reactions.EX_glc__D_e.flux 
    max_yield = nmn_production / (-1*glc_uptake_flux)
    print('Maximum theoretical yield =', max_yield, 'mmol-NMN/mmol-glc') 
    
with nr_model:
    print('With NR pathway:')
    nr_model.objective = nr_model.reactions.FtNadE #Setting the objective to the reaction, which is producing NMN
    nmn_production = nr_model.optimize().objective_value
    print('Maximum productivity of NMN =', nmn_production, 'mmol/gDW*h')

    glc_uptake_flux = nr_model.reactions.EX_glc__D_e.flux 
    max_yield = nmn_production / (-1*glc_uptake_flux)
    print('Maximum theoretical yield =', max_yield, 'mmol-NMN/mmol-glc')

With NAM pathway:
Maximum productivity of NMN = 212.50666666666754 mmol/gDW*h
Maximum theoretical yield = 21.250666666666753 mmol-NMN/mmol-glc
With NR pathway:
Maximum productivity of NMN = 380.5361185983827 mmol/gDW*h
Maximum theoretical yield = 38.05361185983827 mmol-NMN/mmol-glc


### Error with the NA pathway
When FBA is run with the NA with the objective of optimal NMN production it crashes the kernel.

### Optimize for NMN production with cell growth
A double optimization with NMN production as the main target and with still some biomass, so to not kill the cell.

In [7]:
# Run double optimization, both biomass and product.
with nam_model:
    
    # Objective set to NMN
    nam_model.objective = nam_model.reactions.NAMPT
    solution = nam_model.optimize()
    
    # Objective set to Biomass
    nam_model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound=0.5
    nam_model.objective = nam_model.reactions.BIOMASS_Ec_iML1515_core_75p37M
    growth = nam_model.optimize().objective_value
    
    print('Max. NMN production:', solution.fluxes['NAMPT'],'[mmol gDW^-1 h^-1]')
    print('Biomass growth:',nr_model.optimize().fluxes['BIOMASS_Ec_iML1515_core_75p37M'],'[mmol gDW^-1 h^-1]')



Max. NMN production: 212.50666666666754 [mmol gDW^-1 h^-1]
Biomass growth: 14.48258966730976 [mmol gDW^-1 h^-1]


In [8]:
with nr_model:
    
    # Objective set to NMN
    nr_model.objective = na_model.reactions.FtNadE
    solution = na_model.optimize()
    
    # Objective set to Biomass
    nr_model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound
    nr_model.objective = nr_model.reactions.BIOMASS_Ec_iML1515_core_75p37M
    growth = nr_model.optimize().objective_value
    
    print('Max. NMN production:', solution.fluxes['FtNadE'],'[mmol gDW^-1 h^-1]')
    print('Biomass growth:',nr_model.optimize().fluxes['BIOMASS_Ec_iML1515_core_75p37M'],'[mmol gDW^-1 h^-1]')


Max. NMN production: 0.0 [mmol gDW^-1 h^-1]
Biomass growth: 14.48258966730976 [mmol gDW^-1 h^-1]


### For 4. Computer-Aided Cell Factory Engineering (<1500 words if Category II project; <500 words for Category I project)
The 3 pathways were recreated and made in to 3 seperate models called nam_model, na_model and nr_model using the the 3 scripts NAM_pathway, NA_pathway and NR_pathway. The metabolites that took part in the reactions in the 3 pathways that were not present in iML1515 were added to the model. After the reactions were added to each pathway and these could be called for further engineering and Flux Balance Analysis.

For the reactions to take place the medium also had to be altered as these pathways testes by Huang et al. used NAM, NA and NR as substrate which were not in the model medium already. Table 1 shows the medium composition after the additions which shows a that they are present in the model.

For the 3 pathways a couple of knockout were also done to resemble the pathways from Huang et al., but a screening of the model showed that only one of the knocked out genes and the corresponding reactions were present in iML1515. These were knocked out using the knock_out function of cobrapy.

An FBA was run on all 3 models showing the maximum theoretical NMN production and the maximum theoretical yield with regards to the carbon source which in this simulation is glucose.

For the 3 pathways and FSEOF and phase plane were tried to be run but the result were difficult to interpret.

### Discussion
For the productivity and yields of the 3 pathways added to the model we got some results that were difficult to understand. First of all the biomass growth for the 3 models was quite different from each other when running an FBA. The NR pathway shoed more than a 10-fold increase compared to the NAM and NA pathways. This is somewhat unrealistic when comparing the similarity of the NA and NR models. This also takes us to the maxiumum productivity and yield which is extremly high compared to the biomass growth of the cell. Yields of 21 mmol NMN/mmol glucose and 38 mmol NMN/mmol glucose for the NAM an NR pathways respectively is unrealistically high. For the NA pathway the FBA caused a kernel crash when trying to optimize for NMN production, which we were not able to fix as it didn't give any kind of error code. We also tried to do a double optimization the FBA objective was set as NMN production but, without the total loss of biomass. For the NAM pathway this result showed that the maximum production was the same as the theoretical maxiumum and a biomass growth exceeding the theoretical maximum running an FBA solely on biomass. The NR pathway showed no NMN production and the biomass growth was at the same theoretical high. The unrealistic FBA results and the kernel error implies that there may be trouble with the new pathways, which we are not able to discover and fix ourselves.

The substrate for the 3 pathways also seem to not have been added in incorrectly. The cobra documentation does not state any way that has functioned for us, and multiple discussion forums also imply that there seem to be an error when adding new substrate, but not editing excisting.